## CS431/631 Data Intensive Distributed Analytics
### Winter 2020 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Kamran Karim_
* **ID:** _20594936_

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located.   You will be using the Spark installation in the CS451 course account.   The code in the following cell tells Python how to find this Spark installation.   Before going on, execute that code (by selecting the cell and hitting 'return' while holding down the shift key).   It will take a few seconds to run, and will produce no output.

In [23]:
import findspark
findspark.init("/u/cs451/packages/spark")

from pyspark import SparkContext, SparkConf

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster (since we do not have a Spark cluster in the CS student computing environment), and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Run the code in the cell below to create a Spark context.   Creating the `SparkContext` causes your Python program (running in this notebook) to prepare to run Spark jobs, and will take a few seconds to complete.  Be sure that you run this code only one time, because a single Python program may only have one active SparkContext.

In [ ]:
sc = SparkContext(appName="YourTest", master="local[2]")

Next, let's test that your `SparkContext` has been set up properly by running some simple test code (adapted from the [Spark examples page](https://spark.apache.org/examples.html)).   This code uses a single Spark job to estimate the value of $\pi$.  `parallelize()` and `filter()` are Spark *transformations*, and `count()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take a few seconds, since a Spark job is being created and executed, and should print an estimate of $\pi$ when it finishes.   

In [200]:
import random

num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi_estimate = 4 * count / num_samples
print(pi_estimate)

3.14170616


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the $\pi$-estimation example works.   What is the Spark job doing, and how is it used to estimate the value of $\pi$?

#### Your answer to Question 1:

The pi estimation is a simple function that is multiplying the quantity count by a constant 4 and divide the whole multiple by a constant represented with the symbol num_samples (or one can say the division happens first and than the multiplication happens, but it does not change the final answer). 
Spark is involved with calculating the quantity count. 

Firstly, a range function creates an array of integers taking values from 0 to (num_samples-1). Then this array is parallelized (converterd to an RDD), i.e. it allows the spark to distribute the array across multiple nodes using the function "sc.parallelize". After converting the array into an RDD, a filter is applied on this RDD, where the function defined earlier is used. This function takes a dummy input that is the member of the RDD, and generates two random numbers uniformly from between the interval [0,1], and checks if the sum of squares of these two numbers is less than 1 or not. If it is smaller than 1 than the filter keeps the value from RDD, otherwise it discards it. After completing the implementation of this filter, we have a new RDD, and the function ".count()" is applied on it to see how many elements does it have. and finally, this number is the value for the variable count, that we referred earlier.

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.


---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [201]:
#from simple_tokenize import simple_tokenize
lines = sc.textFile("Shakespeare.txt")
words = lines.flatMap(simple_tokenize).cache()
# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens(list = words):
    d = list.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
    count =  d.count()
    return count
    # your solution to Question 2 here
    

In [202]:
count_distinct_tokens()

25975

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [203]:
def pair(s):
    a= simple_tokenize(s)
    l = list()
    for i in a:
        for j in a:
            if j != i:
                if ((i,j),1) not in l:
                    l.append(((i,j),1))
    return l


# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
    d= sc.textFile("Shakespeare.txt").flatMap(pair).reduceByKey(lambda x,y:x+y)
    
    return d.count()
    
    # your solution to Question 3 here


In [204]:
count_distinct_pairs()


1969760

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [205]:
from simple_tokenize import simple_tokenize


def n_line(s):
    a= simple_tokenize(s)
    l = list()
    for i in a:
        if (i,1) not in l:
            l.append((i,1))
    return l
# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
    
    tot = sc.textFile("Shakespeare.txt").count()
    d = sc.textFile("Shakespeare.txt").flatMap(n_line).reduceByKey(lambda x,y:x+y)
    d = d.map(lambda x: ((x[1], x[0]), x[1]/tot)).sortByKey(False).map(lambda x: (x[1], x[0][0], x[0][1])) 
    return d.take(50)
    # your solution to Question 4 here    


In [206]:
top_50_tokens_probabilities()

[(0.2009178657172255, 24604, 'and'),
 (0.19843538192686472, 24300, 'the'),
 (0.1523542765682928, 18657, 'i'),
 (0.148924529226347, 18237, 'to'),
 (0.1357526662202551, 16624, 'of'),
 (0.10844534452628657, 13280, 'a'),
 (0.09959332995802643, 12196, 'you'),
 (0.09430988583840991, 11549, 'my'),
 (0.08667461497003054, 10614, 'in'),
 (0.08630714204053634, 10569, 'that'),
 (0.07150206601447026, 8756, 'is'),
 (0.06720671577193814, 8230, 'not'),
 (0.06167012363422561, 7552, 'with'),
 (0.06039621747864574, 7396, 'me'),
 (0.05982459292165477, 7326, 'for'),
 (0.05836286726877787, 7147, 'it'),
 (0.054402325695340446, 6662, 'be'),
 (0.05246696826667102, 6425, 'this'),
 (0.05228731483447386, 6403, 'his'),
 (0.050899083767495794, 6233, 'your'),
 (0.05067043394469941, 6205, 'but'),
 (0.0474938346208496, 5816, 'he'),
 (0.046889545803459144, 5742, 'have'),
 (0.04132028940534714, 5060, 'thou'),
 (0.04015254209606559, 4917, 'as'),
 (0.03952375508337552, 4840, 'him'),
 (0.03949109082297604, 4836, 'so'),
 (0

---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

Note: the definition of PMI is taken as 
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x,y) = \frac{c(x,y)}{N} $ and $p(x) = \frac{c(x)}{N}$ where $c(x)$ is the number of lines x is seen, similarly $c(x,y)$ is the number of lines x and y co-occur on one line, and finally N is the total number of lines in that document. 

In [207]:
from simple_tokenize import simple_tokenize
from math import log

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)

def PMI(T):
    tot = sc.textFile("Shakespeare.txt").count()
    a0 = sc.textFile("Shakespeare.txt").flatMap(n_line).reduceByKey(lambda x,y:x+y).filter(lambda x: x[1]>T)
    a1 = sc.textFile("Shakespeare.txt").flatMap(pair).reduceByKey(lambda x,y:x+y).filter(lambda x: x[1]>T).map(lambda x: (x[0][0], (x[0][1], x[1])))
    a2 = a1.join(a0)
    a3 = a2.map(lambda x: (x[1][0][0], ((x[0], x[1][0][1]),x[1][1]) ))
    a4 = a3.join(a0)
    a = a4.map(lambda x: ((x[0], x[1][0][0][0]), log(((x[1][0][0][1] *tot)/(x[1][1]*x[1][0][1])), 10) , x[1][0][0][1], x[1][1],x[1][0][1]))

    return a.collect()
              
    # your solution to Question 5 here


In [208]:
#test code here#
#PMI(4000)


[(('of', 'the'), 0.34294075191889306, 7266, 16624, 24300),
 (('and', 'the'), 0.0459349918330654, 5427, 24604, 24300),
 (('to', 'the'), 0.05123525982989819, 4072, 18237, 24300),
 (('the', 'of'), 0.34294075191889306, 7266, 24300, 16624),
 (('the', 'and'), 0.0459349918330654, 5427, 24300, 24604),
 (('the', 'to'), 0.05123525982989819, 4072, 24300, 18237)]

#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```


In [212]:
from simple_tokenize import simple_tokenize
from math import log
import numpy


#takeSample(self, withReplacement, num, seed=None)



# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(T, samp_size):
    a0 = sc.textFile("Shakespeare.txt").flatMap(n_line).reduceByKey(lambda x,y:x+y).filter(lambda x: x[1]>T)
    a1 = sc.textFile("Shakespeare.txt").flatMap(pair).reduceByKey(lambda x,y:x+y).filter(lambda x: x[1]>T).map(lambda x: (x[0][0], (x[0][1], x[1])))
    a1 = a1.reduceByKey(lambda x,y:x+y)

    b = sc.parallelize(a1.takeSample(False, samp_size, None))
    b = b.map(lambda x: (x[0], ""))
    c = sc.parallelize(PMI(2)).filter(lambda x: x[2] > T).map(lambda x: (x[0][0], ( (x[0][0], x[0][1]), x[1], x[2], x[3], x[4])))
    c = c.reduceByKey(lambda x,y:x+y)
    d = c.join(b)

    d = d.map(lambda x: (x[0], [ x[1][0] ]))
    return d.collect()
    


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.

In [213]:
PMI_one_token(2000, 2)

[('you',
  [(('you', 'i'),
    0.18638190375571087,
    2854,
    12196,
    18657,
    ('you', 'to'),
    0.12767187759088927,
    2437,
    12196,
    18237,
    ('you', 'and'),
    -0.05963472348492286,
    2136,
    12196,
    24604)]),
 ('am', [(('am', 'i'), 0.8031984391122662, 2053, 2120, 18657)])]